In [172]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [173]:
print("primeiras linhas do dataframe")

# Definindo o tamanho do chunk para x linhas
chunksize = 20
reader = pd.read_csv('Quali_20240601.csv', chunksize=chunksize)

# Pegando o primeiro chunk
chunk = next(reader)

# Exibindo o chunk
print(chunk)

primeiras linhas do dataframe
                   Legal Fund Name Including Umbrella Fund Code Citi Code  \
0                                                C 11     00F9A      009A   
1                                                C 11     00F9A      009A   
2                                            R-VIP 50     00F9G      009G   
3                                            R-VIP 50     00F9G      009G   
4                                           R-VIP 100     00F9I      009I   
5                                           R-VIP 100     00F9I      009I   
6                                    Raiffeisen R 135     00F9K      009K   
7                                    Raiffeisen R 135     00F9K      009K   
8            JPMorgan Funds - Europe Equity Plus Fund     AEFD9      00A2   
9            JPMorgan Funds - Europe Equity Plus Fund     AEFD9      00A2   
10  Principal Global Investors Funds - Preferred S...     WVF70      00AL   
11  Principal Global Investors Funds - Preferr

In [174]:
with open("colunas_especificas.txt", 'r') as fin:
    cols = fin.readlines()
    cols_especificas = []
    for c in cols:
        cols_especificas.append(c.replace("\n",""))


print(cols_especificas)


['Legal Fund Name Including Umbrella', 'Fund Code', 'Citi Code', 'ISIN', 'WKN', '20000_Financial_Instrument_Identifying_Data', 'EPT Portfolio Name', 'EPT Portfolio or Share Class Currency', 'EPT PRIIPs KID Publication Date', 'EPT Reference Language', 'EPT Valuation Frequency', 'EPT Summary Risk Indicator', 'EPT Is SRI Adjusted', 'EPT Market Risk Measure', 'EPT Credit Risk Measure', 'EPT Recommended Holding Period', 'EPT Number Of Observed Returns', 'EPT Reference Invested Amount', 'EPT Ongoing Costs Management Fees and Other Administrative or Operating Costs', 'EPT Portfolio Transaction Costs', 'EPT Target Market Retail Investor Type', 'EPT Investment Objective', 'EPT Risk Narrative', 'EPT Other Risk Narrative', 'EPT Investment Option', 'Portfolio Turnover Ratio', 'Portfolio Turnover Ratio Date', 'FE Group Name', 'Unit Status', 'Company Contact Address', 'Company Contact E-Mail', 'Company Contact Post Code', 'Company Contact Telephone', 'Company Contact Web Address', 'FE Fund Name', 'S

In [175]:
# Defina o tamanho do chunk para ler um número específico de linhas
nrows_to_read = 100  # Número específico de linhas a serem lidas de cada arquivo

# Dicionário para armazenar os DataFrames processados e a contagem de linhas
chunks_dict = {}
total_lines_dict = {}  # Novo dicionário para armazenar o número total de linhas

for xx in range(1, 31):
    # Formatar o número com dois dígitos
    xx_formatted = f"{xx:02d}"
    filename = f"Quali_202406{xx_formatted}.csv"
    
    print(f"Lendo o arquivo {filename}, limitando a {nrows_to_read} linhas")
    
    try:
        # Ler apenas um número específico de linhas do arquivo atual
        chunk = pd.read_csv(filename, nrows=nrows_to_read)
        
        # Somar o número de linhas lidas no chunk atual
        total_lines = len(chunk)
        
        # Verificar quais colunas estão presentes no chunk
        cols_to_keep = [col for col in cols_especificas if col in chunk.columns]
        
        # Se nenhuma coluna for encontrada, pular o arquivo
        if not cols_to_keep:
            print(f"Nenhuma das colunas especificadas foi encontrada em {filename}")
            continue
        
        # Filtrar as colunas específicas
        chunk_filtered = chunk[cols_to_keep]
        
        # Armazenar o chunk filtrado no dicionário
        chunks_dict[f"chunk_{xx_formatted}"] = chunk_filtered
        
        # Armazenar o número total de linhas lidas para o arquivo atual
        total_lines_dict[f"file_{xx_formatted}"] = total_lines
        print(f"Total de {total_lines} linhas processadas para o arquivo {filename}")
    
    except FileNotFoundError:
        print(f"O arquivo {filename} não foi encontrado. Pulando para o próximo arquivo.")
        continue

# Agora você pode acessar o número total de linhas processadas para cada arquivo
# Por exemplo, para o arquivo do dia 01 de junho:
# print(total_lines_dict['file_01'])


Lendo o arquivo Quali_20240601.csv, limitando a 100 linhas
Total de 100 linhas processadas para o arquivo Quali_20240601.csv
Lendo o arquivo Quali_20240602.csv, limitando a 100 linhas
Total de 100 linhas processadas para o arquivo Quali_20240602.csv
Lendo o arquivo Quali_20240603.csv, limitando a 100 linhas


PermissionError: [Errno 13] Permission denied: 'Quali_20240603.csv'

In [155]:
df_chunk_01 = chunks_dict['chunk_01']
print(df_chunk_01.columns)

Index(['Legal Fund Name Including Umbrella', 'Fund Code', 'Citi Code', 'ISIN',
       'WKN', '20000_Financial_Instrument_Identifying_Data',
       'EPT Portfolio Name', 'EPT Portfolio or Share Class Currency',
       'EPT PRIIPs KID Publication Date', 'EPT Reference Language',
       ...
       'ManCo', 'Domicile Of ManCo', 'Fund Administrator Name',
       'Custodian Bank Name', 'Portfolio Managing Company Name',
       'Transfer Agent Name', 'Auditor Name', 'Fund Promoter Name',
       'Exit Charge', 'Is ETF'],
      dtype='object', length=114)


In [151]:
# Supondo que você já tem o dicionário 'chunks_dict' com os DataFrames para cada dia
# E que as colunas identificadoras são 'ISIN', 'Citi Code', 'EPT Reference Language'

# Lista para armazenar os DataFrames com a coluna 'data' adicionada
dfs_with_date = []

for xx in range(1, 31):
    xx_formatted = f"{xx:02d}"
    key = f"chunk_{xx_formatted}"
    if key in chunks_dict:
        df = chunks_dict[key].copy()
        
        # Adiciona a coluna 'data' com o valor do dia no formato 'xx/06/2024'
        df['data'] = f"{xx_formatted}/06/2024"
        dfs_with_date.append(df)

# Combina todos os DataFrames em um único DataFrame
combined_df = pd.concat(dfs_with_date, ignore_index=True)

# Remove as linhas que possuem valores NaN
combined_df.dropna(inplace=True)

# Ordena o DataFrame pelos identificadores e pela data
combined_df.sort_values(by=['ISIN', 'Citi Code', 'EPT Reference Language', 'data'], inplace=True)

# Lista para armazenar as mudanças detectadas
changes = []

# Definir as colunas identificadoras e as colunas a serem comparadas
id_cols = ['ISIN', 'Citi Code', 'EPT Reference Language']
value_cols = [col for col in combined_df.columns if col not in id_cols + ['data']]

# Agrupa o DataFrame pelos identificadores
grouped = combined_df.groupby(id_cols)

for name, group in grouped:
    group = group.sort_values(by='data')
    group = group.reset_index(drop=True)
    for i in range(1, len(group)):
        current_row = group.loc[i]
        previous_row = group.loc[i-1]
        data_current = current_row['data']  # Data do valor novo
        data_previous = previous_row['data']  # Data do valor antigo
        for col in value_cols:
            value_current = current_row[col]
            value_previous = previous_row[col]
            # Verifica se os valores são diferentes, considerando valores nulos
            if pd.isnull(value_current) and pd.isnull(value_previous):
                continue  # Ambos são nulos, não há mudança
            elif value_current != value_previous:
                changes.append({
                    'ISIN': current_row['ISIN'],
                    'Citi Code': current_row['Citi Code'],
                    'EPT Reference Language': current_row['EPT Reference Language'],
                    'campo': col,
                    'valor_antigo': value_previous,
                    'valor_novo': value_current,
                    'data_antigo': data_previous,
                    'data_novo': data_current
                })

# Cria um DataFrame com as mudanças
changes_df = pd.DataFrame(changes)

# Exibe o DataFrame com as mudanças
print(changes_df)


Empty DataFrame
Columns: []
Index: []


In [152]:
# Seleciona apenas as colunas desejadas e imprime o DataFrame
#print(changes_df[['campo', 'valor_antigo', 'valor_novo', 'data']])

In [153]:
# Supondo que você já tem o dicionário 'chunks_dict' com os DataFrames para cada dia
# E que as colunas identificadoras são 'ISIN', 'Citi Code', 'EPT Reference Language'

# Lista para armazenar os DataFrames com a coluna 'data' adicionada
dfs_with_date = []

for xx in range(1, 31):
    xx_formatted = f"{xx:02d}"
    key = f"chunk_{xx_formatted}"
    if key in chunks_dict:
        df = chunks_dict[key].copy()
        # Adiciona a coluna 'data' com o valor do dia no formato 'xx/06/2024'
        df['data'] = f"{xx_formatted}/06/2024"
        dfs_with_date.append(df)

# Combina todos os DataFrames em um único DataFrame
combined_df = pd.concat(dfs_with_date, ignore_index=True)

# Converter a coluna 'data' para datetime para facilitar a ordenação
combined_df['data'] = pd.to_datetime(combined_df['data'], format='%d/%m/%Y')

# Ordena o DataFrame pelos identificadores e pela data
combined_df.sort_values(by=['ISIN', 'Citi Code', 'EPT Reference Language', 'data'], inplace=True)

# Lista para armazenar as mudanças detectadas
changes = []

# Definir as colunas identificadoras e as colunas a serem comparadas
id_cols = ['ISIN', 'Citi Code', 'EPT Reference Language']
value_cols = [col for col in combined_df.columns if col not in id_cols + ['data']]

# Agrupa o DataFrame pelos identificadores
grouped = combined_df.groupby(id_cols)

for name, group in grouped:
    group = group.sort_values(by='data')
    group = group.reset_index(drop=True)
    for i in range(1, len(group)):
        current_row = group.loc[i]
        previous_row = group.loc[i-1]
        data_current = current_row['data'].strftime('%d/%m/%Y')  # Converte para string no formato desejado
        data_previous = previous_row['data'].strftime('%d/%m/%Y')
        for col in value_cols:
            value_current = current_row[col]
            value_previous = previous_row[col]
            # Verifica se os valores são diferentes, considerando valores nulos
            if pd.isnull(value_current) and pd.isnull(value_previous):
                continue  # Ambos são nulos, não há mudança
            elif pd.isnull(value_current) != pd.isnull(value_previous):
                # Um valor é nulo e o outro não
                changes.append({
                    'campo': col,
                    'valor_antigo': value_previous,
                    'valor_novo': value_current,
                    'data': data_current
                })
            elif value_current != value_previous:
                changes.append({
                    'campo': col,
                    'valor_antigo': value_previous,
                    'valor_novo': value_current,
                    'data': data_current
                })

# Cria um DataFrame com as mudanças
changes_df = pd.DataFrame(changes)

# Seleciona apenas as colunas desejadas
changes_df = changes_df[['campo', 'valor_antigo', 'valor_novo', 'data']]

# Converter a coluna 'data' para datetime para ordenação
changes_df['data'] = pd.to_datetime(changes_df['data'], format='%d/%m/%Y')

# Ordena o DataFrame 'changes_df' pela coluna 'data'
changes_df.sort_values(by='data', inplace=True)

# Opcional: Converte a coluna 'data' de volta para string no formato desejado
changes_df['data'] = changes_df['data'].dt.strftime('%d/%m/%Y')

# Salva o DataFrame 'changes_df' em um arquivo CSV
changes_df.to_csv('mudancas_detectadas.csv', index=False)

# Exibe o DataFrame com as mudanças ordenadas por data
print(changes_df)


                                          campo  valor_antigo    valor_novo  \
0   20000_Financial_Instrument_Identifying_Data  AT0000691365           NaN   
21  20000_Financial_Instrument_Identifying_Data  LU0289214545           NaN   
3   20000_Financial_Instrument_Identifying_Data  AT0000704366           NaN   
18  20000_Financial_Instrument_Identifying_Data  IE00B61TKZ25           NaN   
6   20000_Financial_Instrument_Identifying_Data  AT0000721402           NaN   
9   20000_Financial_Instrument_Identifying_Data  AT0000779392           NaN   
15  20000_Financial_Instrument_Identifying_Data  AT0000A0FA24           NaN   
12  20000_Financial_Instrument_Identifying_Data  AT0000A0F9X2           NaN   
20  20000_Financial_Instrument_Identifying_Data  IE00B61TKZ25           NaN   
19  20000_Financial_Instrument_Identifying_Data           NaN  IE00B61TKZ25   
17  20000_Financial_Instrument_Identifying_Data  AT0000A0FA24           NaN   
16  20000_Financial_Instrument_Identifying_Data     

In [154]:
print(changes_df[['data']])

          data
0   01/06/2024
21  01/06/2024
3   01/06/2024
18  01/06/2024
6   01/06/2024
9   01/06/2024
15  01/06/2024
12  01/06/2024
20  02/06/2024
19  02/06/2024
17  02/06/2024
16  02/06/2024
14  02/06/2024
11  02/06/2024
22  02/06/2024
10  02/06/2024
8   02/06/2024
7   02/06/2024
5   02/06/2024
4   02/06/2024
2   02/06/2024
1   02/06/2024
13  02/06/2024
23  02/06/2024
